In [1]:
from perses.utils.openeye import *
from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
RDKit WARNING: [17:44:28] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']


In [3]:
# First thing to do is load the apo protein to mutate...
protein_filename = '../input/ala_vacuum.pdb'
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms

In [5]:
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
periodic_forcefield_kwargs = None
nonperiodic_forcefield_kwargs = nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}
molecules = []
system_generator = SystemGenerator(forcefields = forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                   nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                molecules=molecules,
                                                cache=None)

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load openff-1.0.0
DEBUG:perses.forcefields.system_generators:  GAFFTemplateGenerator cannot load openff-1.0.0
DEBUG:perses.forcefields.system_generators:Specified 'forcefield' (openff-1.0.0) must be one of ['gaff-1.4', 'gaff-1.8', 'gaff-1.81', 'gaff-2.1', 'gaff-2.11']
DEBUG:perses.forcefields.system_generators:Trying SMIRNOFFTemplateGenerator to load openff-1.0.0


In [6]:
def _solvate(topology,
                    positions,
                    model,
                    phase):
    """
    Generate a solvated topology, positions, and system for a given input topology and positions.
    For generating the system, the forcefield files provided in the constructor will be used.

    Parameters
    ----------
    topology : app.Topology
        Topology of the system to solvate
    positions : [n, 3] ndarray of Quantity nm
        the positions of the unsolvated system
    forcefield : SystemGenerator.forcefield
        forcefield file of solvent to add
    model : str, default 'tip3p'
        solvent model to use for solvation

    Returns
    -------
    solvated_topology : app.Topology
        Topology of the system with added waters
    solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
        Solvated positions
    solvated_system : openmm.System
        The parameterized system, containing a barostat if one was specified.
    """
    modeller = app.Modeller(topology, positions)


    # Now we have to add missing atoms
    if phase != 'vacuum':
        modeller.addSolvent(system_generator.forcefield, model=model, padding=1.0 * unit.nanometers, ionicStrength=0.05*unit.molar)
    else:
        pass

    solvated_topology = modeller.getTopology()
    solvated_positions = modeller.getPositions()

    # Canonicalize the solvated positions: turn tuples into np.array
    solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    solvated_system = system_generator.create_system(solvated_topology)

    return solvated_topology, solvated_positions, solvated_system


In [40]:
top, pos, sys = list(_solvate(protein_topology, protein_positions, 'tip3p', phase='vacuum'))

In [41]:
outdir = "/home/zhangi/choderalab/perses/examples/protein-atom-mapping"

In [51]:
import os
import pickle
with open(os.path.join(outdir, f"ala_ser_top.pickle"), 'wb') as f:
    pickle.dump(top, f)
with open(os.path.join(outdir, f"ala_ser_pos.pickle"), 'wb') as f:
    pickle.dump(pos, f)
with open(os.path.join(outdir, f"ala_ser_sys.pickle"), 'wb') as f:
    pickle.dump(sys, f)
with open(os.path.join(outdir, f"system_generator.pickle"), 'wb') as f:
    pickle.dump(system_generator, f)